# **Laboratorio 12: 🚀 Despliegue 🚀**

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Israel Astudillo M.
- Nombre de alumno 2: Luis Picón

### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/IsraPKMNPAP/Laboratorio-de-Herramientas)

## Temas a tratar

- Entrenamiento y registro de modelos usando MLFlow.
- Despliegue de modelo usando FastAPI
- Containerización del proyecto usando Docker

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Generar una solución a un problema a partir de ML
- Desplegar su solución usando MLFlow, FastAPI y Docker

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# **Introducción**

<p align="center">
  <img src="https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExODJnMHJzNzlkNmQweXoyY3ltbnZ2ZDlxY2c0aW5jcHNzeDNtOXBsdCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/AbPdhwsMgjMjax5reo/giphy.gif" width="400">
</p>



Consumida en la tristeza el despido de Renacín, Smapina ha decaído en su desempeño, lo que se ha traducido en un irregular tratamiento del agua. Esto ha implicado una baja en la calidad del agua, llegando a haber algunos puntos de la comuna en la que el vital elemento no es apto para el consumo humano. Es por esto que la sanitaria pública de la municipalidad de Maipú se ha contactado con ustedes para que le entreguen una urgente solución a este problema (a la vez que dejan a Smapina, al igual que Renacín, sin trabajo 😔).

El problema que la empresa le ha solicitado resolver es el de elaborar un sistema que les permita saber si el agua es potable o no. Para esto, la sanitaria les ha proveido una base de datos con la lectura de múltiples sensores IOT colocados en diversas cañerías, conductos y estanques. Estos sensores señalan nueve tipos de mediciones químicas y más una etiqueta elaborada en laboratorio que indica si el agua es potable o no el agua.

La idea final es que puedan, en el caso que el agua no sea potable, dar un aviso inmediato para corregir el problema. Tenga en cuenta que parte del equipo docente vive en Maipú y su intoxicación podría implicar graves problemas para el cierre del curso.

Atributos:

1. pH value
2. Hardness
3. Solids (Total dissolved solids - TDS)
4. Chloramines
5. Sulfate
6. Conductivity
7. Organic_carbon
8. Trihalomethanes
9. Turbidity

Variable a predecir:

10. Potability (1 si es potable, 0 no potable)

Descripción de cada atributo se pueden encontrar en el siguiente link: [dataset](https://www.kaggle.com/adityakadiwal/water-potability)

# **1. Optimización de modelos con Optuna + MLFlow (2.0 puntos)**

El objetivo de esta sección es que ustedes puedan combinar Optuna con MLFlow para poder realizar la optimización de los hiperparámetros de sus modelos.

Como aún no hemos hablado nada sobre `MLFlow` cabe preguntarse: **¡¿Qué !"#@ es `MLflow`?!**

<p align="center">
  <img src="https://media.tenor.com/eusgDKT4smQAAAAC/matthew-perry-chandler-bing.gif" width="400">
</p>

## **MLFlow**

`MLflow` es una plataforma de código abierto que simplifica la gestión y seguimiento de proyectos de aprendizaje automático. Con sus herramientas, los desarrolladores pueden organizar, rastrear y comparar experimentos, además de registrar modelos y controlar versiones.

<p align="center">
  <img src="https://spark.apache.org/images/mlflow-logo.png" width="350">
</p>

Si bien esta plataforma cuenta con un gran número de herramientas y funcionalidades, en este laboratorio trabajaremos con dos:
1. **Runs**: Registro que constituye la información guardada tras la ejecución de un entrenamiento. Cada `run` tiene su propio run_id, el cual sirve como identificador para el entrenamiento en sí mismo. Dentro de cada `run` podremos acceder a información como los hiperparámetros utilizados, las métricas obtenidas, las librerías requeridas y hasta nos permite descargar el modelo entrenado.
2. **Experiments**: Se utilizan para agrupar y organizar diferentes ejecuciones de modelos (`runs`). En ese sentido, un experimento puede agrupar 1 o más `runs`. De esta manera, es posible también registrar métricas, parámetros y archivos (artefactos) asociados a cada experimento.

### **Todo bien pero entonces, ¿cómo se usa en la práctica `MLflow`?**

Es sencillo! Considerando un problema de machine learning genérico, podemos registrar la información relevante del entrenamiento ejecutando `mlflow.autolog()` antes entrenar nuestro modelo. Veamos este bonito ejemplo facilitado por los mismos creadores de `MLflow`:

```python
#!pip install mlflow
import mlflow # importar mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor

db = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(db.data, db.target)

# Create and train models.
rf = RandomForestRegressor(n_estimators=100, max_depth=6, max_features=3)

mlflow.autolog() # registrar automáticamente información del entrenamiento
with mlflow.start_run(): # delimita inicio y fin del run
    # aquí comienza el run
    rf.fit(X_train, y_train) # train the model
    predictions = rf.predict(X_test) # Use the model to make predictions on the test dataset.
    # aquí termina el run
```

Si ustedes ejecutan el código anterior en sus máquinas locales (desde un jupyter notebook por ejemplo) se darán cuenta que en su directorio *root* se ha creado la carpeta `mlruns`. Esta carpeta lleva el tracking de todos los entrenamientos ejecutados desde el directorio root (importante: si se cambian de directorio y vuelven a ejecutar el código anterior, se creará otra carpeta y no tendrán acceso al entrenamiento anterior). Para visualizar estos entrenamientos, `MLflow` nos facilita hermosa interfaz visual a la que podemos acceder ejecutando:

```
mlflow ui
```

y luego pinchando en la ruta http://127.0.0.1:5000 que nos retorna la terminal. Veamos en vivo algunas de sus funcionalidades!

<p align="center">
  <img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExZXVuM3A5MW1heDFpa21qbGlwN2pyc2VoNnZsMmRzODZxdnluemo2bCZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/3o84sq21TxDH6PyYms/giphy.gif" width="400">
</p>

Les dejamos también algunos comandos útiles:

- `mlflow.create_experiment("nombre_experimento")`: Les permite crear un nuevo experimento para agrupar entrenamientos
- `mlflow.log_metric("nombre_métrica", métrica)`: Les permite registrar una métrica *custom* bajo el nombre de "nombre_métrica"


## **1.1 Combinando Optuna + MLflow (2.0 puntos)**

Ahora que tenemos conocimiento de ambas herramientas, intentemos ahora combinarlas para **más sabor**. El objetivo de este apartado es simple: automatizar la optimización de los parámetros de nuestros modelos usando `Optuna` y registrando de forma automática cada resultado en `MLFlow`.

Considerando el objetivo planteado, se le pide completar la función `optimize_model`, la cual debe:
- **Optimizar los hiperparámetros del modelo `XGBoost` usando `Optuna`.**
- **Registrar cada entrenamiento en un experimento nuevo**, asegurándose de que la métrica `f1-score` se registre como `"valid_f1"`. No se deben guardar todos los experimentos en *Default*; en su lugar, cada `experiment` y `run` deben tener nombres interpretables, reconocibles y diferentes a los nombres por defecto (por ejemplo, para un run: "XGBoost con lr 0.1").
- **Guardar los gráficos de Optuna** dentro de una carpeta de artefactos de Mlflow llamada `/plots`.
- **Devolver el mejor modelo** usando la función `get_best_model` y serializarlo en el disco con `pickle.dump`. Luego, guardar el modelo en la carpeta `/models`.
- **Guardar el código en `optimize.py`**. La ejecución de `python optimize.py` debería ejecutar la función `optimize_model`.
- **Guardar las versiones de las librerías utilizadas** en el desarrollo.
- **Respalde las configuraciones del modelo final y la importancia de las variables** en un gráfico dentro de la carpeta `/plots` creada anteriormente.

*Hint: Le puede ser útil revisar los parámetros que recibe `mlflow.start_run`*

```python
def get_best_model(experiment_id):
    runs = mlflow.search_runs(experiment_id)
    best_model_id = runs.sort_values("metrics.valid_f1")["run_id"].iloc[0]
    best_model = mlflow.sklearn.load_model("runs:/" + best_model_id + "/model")

    return best_model
```

Se adjunta la respuesta primero ejecutable en colab y luego se adjunta la estructura de los archivos pedidos.

In [ ]:
import pandas as pd
# Leemos el dataset
df = pd.read_csv('water_potability.csv')
df.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


In [ ]:
# Tenemos solo columnas numéricas, las cuales son
cols = df.columns.tolist()
print("Columnas del dataset:")
print(cols)
# Notamos que la data tarea vacíos
df.dropna(inplace=True)
print("Revisión de eliminación de nulos:")
print(df.isna().sum())

Columnas del dataset:
['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity', 'Organic_carbon', 'Trihalomethanes', 'Turbidity', 'Potability']
Revisión de eliminación de nulos:
ph                 0
Hardness           0
Solids             0
Chloramines        0
Sulfate            0
Conductivity       0
Organic_carbon     0
Trihalomethanes    0
Turbidity          0
Potability         0
dtype: int64


In [ ]:
!pip install -qq xgboost optuna mlflow plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.1/575.1 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.3 MB/s eta 0:00:00


In [ ]:
!pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.2 MB/s eta 0:00:00


In [ ]:
import xgboost as xgb
print(xgb.__version__)

2.1.2


In [ ]:
!pip install xgboost --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 5.0 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.2
    Uninstalling xgboost-2.1.2:
      Successfully uninstalled xgboost-2.1.2


In [ ]:
# Librerías necesarias
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler

import optuna

import mlflow
import mlflow.xgboost

import os
import pickle
import shutil

import matplotlib.pyplot as plt

In [ ]:
# Configurar directorios
BASE_DIR = "/content/mlflow_experiment"  # Base de MLFlow
OUTPUT_DIR = "/content/outputs"  # Para gráficos y modelos
PLOTS_DIR = os.path.join(OUTPUT_DIR, "plots")
MODELS_DIR = os.path.join(OUTPUT_DIR, "models")
os.makedirs(PLOTS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

In [ ]:
# Configurar el servidor local de MLFlow
MLFLOW_TRACKING_URI = BASE_DIR  # Directorio local para experimentos
EXPERIMENT_NAME = "XGBoost_Optimization_Colab"
mlflow.set_tracking_uri(f"file://{MLFLOW_TRACKING_URI}")
mlflow.set_experiment(EXPERIMENT_NAME)

2024/11/26 15:16:44 INFO mlflow.tracking.fluent: Experiment with name 'XGBoost_Optimization_Colab' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlflow_experiment/885920864976791736', creation_time=1732634204857, experiment_id='885920864976791736', last_update_time=1732634204857, lifecycle_stage='active', name='XGBoost_Optimization_Colab', tags={}>

In [ ]:
# Escalar los datos
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=cols)

# Definir características y objetivo
X = df_scaled.drop('Potability', axis=1)
y = df_scaled['Potability']
# Separar datos en entrenamiento y validación
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def objective(trial):
    num_classes = len(set(y_train))
    # Probar hiperparámetros con Optuna
    params = {
        "objective": "multi:softmax",
        "num_class": num_classes,
        "max_depth": trial.suggest_int("max_depth", 3, 8),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 7),
        "gamma": trial.suggest_float("gamma", 0, 1),
        "n_estimators": trial.suggest_int("n_estimators", 10, 300),
        "use_label_encoder": False,  # Evita warnings en XGBoost
    }

    # Crear modelo
    model = XGBClassifier(**params)

    # Entrenar el modelo con conjunto de validación
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],  # Para seguimiento de métricas en validación
        #early_stopping_rounds=10,
        verbose=False,
    )

    # Evaluar f1-score
    y_pred = model.predict(X_valid)
    f1 = f1_score(y_valid, y_pred, average="weighted")

    # Registrar métrica con MLFlow
    mlflow.log_metric("valid_f1", f1)
    return f1

In [ ]:
from optuna.visualization import plot_param_importances, plot_optimization_history
import kaleido
# Optimizar con Optuna
def optimize():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)

    # Crear directorios si no existen
    os.makedirs(PLOTS_DIR, exist_ok=True)
    os.makedirs(MODELS_DIR, exist_ok=True)

    # Guardar gráficos de Optuna
    param_importance_path = os.path.join(PLOTS_DIR, "param_importances.png")
    optimization_history_path = os.path.join(PLOTS_DIR, "optimization_history.png")

    # Crear y guardar gráficos de visualización de Optuna
    optuna.visualization.plot_param_importances(study).write_image(param_importance_path)
    optuna.visualization.plot_optimization_history(study).write_image(optimization_history_path)

    # Registrar gráficos como artefactos en MLFlow
    mlflow.log_artifact(param_importance_path, artifact_path="plots")
    mlflow.log_artifact(optimization_history_path, artifact_path="plots")

    # Guardar el mejor modelo
    best_params = study.best_params
    best_model = XGBClassifier(
        seed=42,
        use_label_encoder=False,
        **best_params,
    )

    # Ajustar el modelo con los mejores parámetros
    best_model.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],
        #early_stopping_rounds=10,
        verbose=False
    )

    # Serializar y guardar el modelo
    model_path = os.path.join(MODELS_DIR, "best_model.pkl")
    with open(model_path, "wb") as f:
        pickle.dump(best_model, f)

    # Registrar el modelo en MLFlow
    mlflow.log_artifact(model_path, artifact_path="models")

In [ ]:
optimize()

[I 2024-11-26 15:16:46,282] A new study created in memory with name: no-name-056a8960-73fe-4e3f-9406-18b21fd87fe0
<ipython-input-12-c001f3372292>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:16:46] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-11-26 15:16:48,801] Trial 0 finished with value: 0.6367540647937671 and parameters: {'max_depth': 8, 'learning_rate': 0.05003821675287064, 'subsample': 0.8896945070391186, 'colsample_bytree': 0.8956098934203184, 'min_child_weight': 5, 'gamma': 0.654278965921727, 'n_estimators': 81}. Best is trial 0 with value: 0.6367540647937671.
<ipython-input-12-c0

Archivo optimize.py

In [ ]:
# optimize.py

import pandas as pd
# Leemos el dataset
df = pd.read_csv('water_potability.csv')
# Tenemos solo columnas numéricas, las cuales son
cols = df.columns.tolist()
# Notamos que la data tarea vacíos
df.dropna(inplace=True)
# Librerías necesarias
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import optuna
import mlflow
import mlflow.xgboost
import os
import pickle
import shutil
import matplotlib.pyplot as plt
from optuna.visualization import plot_param_importances, plot_optimization_history
import kaleido
# Escalar los datos
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=cols)
# Definir características y objetivo
X = df_scaled.drop('Potability', axis=1)
y = df_scaled['Potability']
# Separar datos en entrenamiento y validación
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
# Directorios
PLOTS_DIR = "plots"
MODELS_DIR = "models"
os.makedirs(PLOTS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)
mlflow.set_tracking_uri("file:./mlruns")
# Objective
def objective(trial):
    num_classes = len(set(y_train))
    # Probar hiperparámetros con Optuna
    params = {
        "objective": "multi:softmax",
        "num_class": num_classes,
        "max_depth": trial.suggest_int("max_depth", 3, 8),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.1),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 7),
        "gamma": trial.suggest_float("gamma", 0, 1),
        "n_estimators": trial.suggest_int("n_estimators", 10, 300),
        "use_label_encoder": False,  # Evita warnings en XGBoost
    }

    # Crear modelo
    model = XGBClassifier(**params)

    # Entrenar el modelo con conjunto de validación
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],  # Para seguimiento de métricas en validación
        #early_stopping_rounds=10,
        verbose=False,
    )

    # Evaluar f1-score
    y_pred = model.predict(X_valid)
    f1 = f1_score(y_valid, y_pred, average="weighted")

    # Registrar métrica con MLFlow
    mlflow.log_metric("valid_f1", f1)
    return f1
# Optimizar con Optuna
def optimize_model():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=50)

    # Crear directorios si no existen
    os.makedirs(PLOTS_DIR, exist_ok=True)
    os.makedirs(MODELS_DIR, exist_ok=True)

    # Guardar gráficos de Optuna
    param_importance_path = os.path.join(PLOTS_DIR, "param_importances.png")
    optimization_history_path = os.path.join(PLOTS_DIR, "optimization_history.png")

    # Crear y guardar gráficos de visualización de Optuna
    optuna.visualization.plot_param_importances(study).write_image(param_importance_path)
    optuna.visualization.plot_optimization_history(study).write_image(optimization_history_path)

    # Registrar gráficos como artefactos en MLFlow
    mlflow.log_artifact(param_importance_path, artifact_path="plots")
    mlflow.log_artifact(optimization_history_path, artifact_path="plots")

    # Guardar el mejor modelo
    best_params = study.best_params
    best_model = XGBClassifier(
        seed=42,
        use_label_encoder=False,
        **best_params,
    )

    # Ajustar el modelo con los mejores parámetros
    best_model.fit(
        X_train,
        y_train,
        eval_set=[(X_valid, y_valid)],
        #early_stopping_rounds=10,
        verbose=False
    )

    # Serializar y guardar el modelo
    model_path = os.path.join(MODELS_DIR, "best_model.pkl")
    with open(model_path, "wb") as f:
        pickle.dump(best_model, f)

    # Registrar el modelo en MLFlow
    mlflow.log_artifact(model_path, artifact_path="models")


if __name__ == "__main__":
    optimize_model()

Archivo requirements.txt

In [ ]:
# requirements.txt

aiohappyeyeballs @ file:///home/conda/feedstock_root/build_artifacts/aiohappyeyeballs_1727779797566/work
aiohttp @ file:///D:/bld/aiohttp_1732220296204/work
aiosignal @ file:///home/conda/feedstock_root/build_artifacts/aiosignal_1667935791922/work
alembic @ file:///home/conda/feedstock_root/build_artifacts/alembic_1730767847981/work
asttokens==2.0.5
async-timeout @ file:///home/conda/feedstock_root/build_artifacts/async-timeout_1730974670620/work
attrs @ file:///home/conda/feedstock_root/build_artifacts/attrs_1722977137225/work
backcall==0.2.0
bcrypt @ file:///D:/bld/bcrypt_1732074871877/work
blinker @ file:///home/conda/feedstock_root/build_artifacts/blinker_1731096409132/work
Brotli @ file:///D:/bld/brotli-split_1725267609074/work
cachetools @ file:///home/conda/feedstock_root/build_artifacts/cachetools_1724028158384/work
certifi @ file:///C:/b/abs_1fw_exq1si/croot/certifi_1725551736618/work/certifi
cffi @ file:///D:/bld/cffi_1725560649097/work
charset-normalizer @ file:///home/conda/feedstock_root/build_artifacts/charset-normalizer_1728479282467/work
click @ file:///D:/bld/click_1692312014553/work
cloudpickle @ file:///home/conda/feedstock_root/build_artifacts/cloudpickle_1729059237860/work
colorama==0.4.4
colorlog @ file:///D:/bld/colorlog_1730249778987/work
contourpy @ file:///D:/bld/contourpy_1731428338157/work
cryptography @ file:///D:/bld/cryptography-split_1729286691806/work
cycler @ file:///home/conda/feedstock_root/build_artifacts/cycler_1696677705766/work
databricks-sdk @ file:///home/conda/feedstock_root/build_artifacts/databricks-sdk_1732030533321/work
debugpy==1.5.1
Deprecated @ file:///home/conda/feedstock_root/build_artifacts/deprecated_1731836826792/work
docker @ file:///home/conda/feedstock_root/build_artifacts/docker-py_1716508870406/work
entrypoints==0.4
executing==0.8.3
Flask @ file:///home/conda/feedstock_root/build_artifacts/flask_1731556349671/work
fonttools @ file:///D:/bld/fonttools_1731643410328/work
frozenlist @ file:///D:/bld/frozenlist_1729699474736/work
gitdb @ file:///home/conda/feedstock_root/build_artifacts/gitdb_1697791558612/work
GitPython @ file:///home/conda/feedstock_root/build_artifacts/gitpython_1711991025291/work
google-auth @ file:///home/conda/feedstock_root/build_artifacts/google-auth_1730952254284/work
graphene @ file:///home/conda/feedstock_root/build_artifacts/graphene_1731260016262/work
graphql-core @ file:///home/conda/feedstock_root/build_artifacts/graphql-core_1728910484470/work
graphql-relay @ file:///home/conda/feedstock_root/build_artifacts/graphql-relay_1650134628625/work
greenlet @ file:///D:/bld/greenlet_1726922269908/work
h2 @ file:///home/conda/feedstock_root/build_artifacts/h2_1634280454336/work
hpack==4.0.0
hyperframe @ file:///home/conda/feedstock_root/build_artifacts/hyperframe_1619110129307/work
idna @ file:///home/conda/feedstock_root/build_artifacts/idna_1726459485162/work
importlib_metadata @ file:///home/conda/feedstock_root/build_artifacts/importlib-metadata_1726082825846/work
importlib_resources @ file:///home/conda/feedstock_root/build_artifacts/importlib_resources_1725921340658/work
ipykernel==6.9.1
ipython==8.1.1
itsdangerous @ file:///home/conda/feedstock_root/build_artifacts/itsdangerous_1713372668944/work
jedi==0.18.1
Jinja2 @ file:///home/conda/feedstock_root/build_artifacts/jinja2_1715127149914/work
joblib @ file:///home/conda/feedstock_root/build_artifacts/joblib_1714665484399/work
jupyter-client==7.1.2
jupyter-core==4.9.2
kaleido==0.2.1
kiwisolver @ file:///D:/bld/kiwisolver_1725459382062/work
Mako @ file:///home/conda/feedstock_root/build_artifacts/mako_1731872826199/work
Markdown @ file:///home/conda/feedstock_root/build_artifacts/markdown_1710435156458/work
MarkupSafe @ file:///D:/bld/markupsafe_1729351293186/work
matplotlib==3.9.2
matplotlib-inline==0.1.3
mlflow-skinny @ file:///D:/bld/mlflow-split_1732046806623/work
multidict @ file:///D:/bld/multidict_1729065633457/work
munkres==1.1.4
nest-asyncio==1.5.4
numpy @ file:///D:/bld/numpy_1730588038333/work/dist/numpy-2.1.3-cp310-cp310-win_amd64.whl#sha256=c29fd581b8df1c3329e6d95e58bdf9db58b0a6a4d713088f166cf377062400db
opentelemetry-api @ file:///home/conda/feedstock_root/build_artifacts/opentelemetry-api_1676680662101/work
opentelemetry-sdk @ file:///home/conda/feedstock_root/build_artifacts/opentelemetry-sdk_1676709164054/work
opentelemetry-semantic-conventions @ file:///home/conda/feedstock_root/build_artifacts/opentelemetry-semantic-conventions_1676680479396/work
optuna @ file:///home/conda/feedstock_root/build_artifacts/optuna_1731474796962/work
packaging @ file:///home/conda/feedstock_root/build_artifacts/packaging_1731802491770/work
pandas @ file:///D:/bld/pandas_1726878561601/work
paramiko @ file:///home/conda/feedstock_root/build_artifacts/paramiko_1726748051454/work
parso==0.8.3
pickleshare==0.7.5
pillow @ file:///D:/bld/pillow_1726075253811/work
plotly @ file:///C:/b/abs_1014knmz1t/croot/plotly_1726245573566/work
prometheus_client @ file:///home/conda/feedstock_root/build_artifacts/prometheus_client_1726901976720/work
prometheus_flask_exporter @ file:///home/conda/feedstock_root/build_artifacts/prometheus_flask_exporter_1720670279306/work
prompt-toolkit==3.0.28
propcache @ file:///D:/bld/propcache_1728545928779/work
protobuf @ file:///D:/bld/protobuf_1728668497421/work/bazel-bin/python/dist/protobuf-5.28.2-cp310-abi3-win_amd64.whl#sha256=fc57b8d440a0b7bf85f818a36f4ec712545512e0a4a83a5349271a0d1c8ecadf
pure-eval==0.2.2
pyarrow==18.1.0
pyasn1 @ file:///home/conda/feedstock_root/build_artifacts/pyasn1_1726839225972/work
pyasn1_modules @ file:///home/conda/feedstock_root/build_artifacts/pyasn1-modules_1726029546107/work
pycparser @ file:///home/conda/feedstock_root/build_artifacts/pycparser_1711811537435/work
Pygments==2.11.2
PyNaCl @ file:///D:/bld/pynacl_1725739406106/work
pyOpenSSL @ file:///home/conda/feedstock_root/build_artifacts/pyopenssl_1722587090966/work
pyparsing @ file:///home/conda/feedstock_root/build_artifacts/pyparsing_1728880423364/work
PySide6==6.8.0.2
PySocks @ file:///D:/bld/pysocks_1661604991356/work
python-dateutil==2.8.2
pytz @ file:///home/conda/feedstock_root/build_artifacts/pytz_1706886791323/work
pyu2f @ file:///home/conda/feedstock_root/build_artifacts/pyu2f_1604248910016/work
pywin32==303
PyYAML @ file:///D:/bld/pyyaml_1725456311802/work
pyzmq==22.3.0
querystring_parser @ file:///home/conda/feedstock_root/build_artifacts/querystring_parser_1723625595981/work
requests @ file:///home/conda/feedstock_root/build_artifacts/requests_1717057054362/work
rsa @ file:///home/conda/feedstock_root/build_artifacts/rsa_1658328885051/work
scikit-learn @ file:///D:/bld/scikit-learn_1726082855864/work/dist/scikit_learn-1.5.2-cp310-cp310-win_amd64.whl#sha256=f8a2b0c9a97f54c5c064931b1b27e9443957ccd063bc8c437288596a61b2bd5d
scipy @ file:///C:/bld/scipy-split_1729480777186/work/dist/scipy-1.14.1-cp310-cp310-win_amd64.whl#sha256=0bd1a786f4784f64d7038cb496e376c7aee37fd463d42853dbee10be0eb97a75
shiboken6==6.8.0.2
six @ file:///home/conda/feedstock_root/build_artifacts/six_1620240208055/work
smmap @ file:///home/conda/feedstock_root/build_artifacts/smmap_1634310307496/work
SQLAlchemy @ file:///D:/bld/sqlalchemy_1729066497613/work
sqlparse @ file:///home/conda/feedstock_root/build_artifacts/sqlparse_1731601717992/work
stack-data==0.2.0
tenacity @ file:///C:/b/abs_1749lsqys6/croot/tenacity_1730304400080/work
threadpoolctl @ file:///home/conda/feedstock_root/build_artifacts/threadpoolctl_1714400101435/work
tornado==6.1
tqdm @ file:///home/conda/feedstock_root/build_artifacts/tqdm_1732497199771/work
traitlets==5.1.1
typing_extensions @ file:///home/conda/feedstock_root/build_artifacts/typing_extensions_1717802530399/work
tzdata @ file:///home/conda/feedstock_root/build_artifacts/python-tzdata_1727140567071/work
unicodedata2 @ file:///D:/bld/unicodedata2_1729704580516/work
urllib3 @ file:///home/conda/feedstock_root/build_artifacts/urllib3_1726496430923/work
waitress @ file:///home/conda/feedstock_root/build_artifacts/waitress_1653960027524/work
wcwidth==0.2.5
websocket-client @ file:///home/conda/feedstock_root/build_artifacts/websocket-client_1713923384721/work
Werkzeug @ file:///home/conda/feedstock_root/build_artifacts/werkzeug_1731097436890/work
win_inet_pton @ file:///D:/bld/win_inet_pton_1727796272493/work
wrapt @ file:///D:/bld/wrapt_1732523626351/work
xgboost @ file:///home/conda/feedstock_root/build_artifacts/xgboost-split_1732150600463/work/python-package
yarl @ file:///D:/bld/yarl_1732220881007/work
zipp @ file:///home/conda/feedstock_root/build_artifacts/zipp_1731262100163/work
zstandard==0.23.0

# **2. FastAPI (2.0 puntos)**

<div align="center">
  <img src="https://media3.giphy.com/media/YQitE4YNQNahy/giphy-downsized-large.gif" width="500">
</div>

Con el modelo ya entrenado, la idea de esta sección es generar una API REST a la cual se le pueda hacer *requests* para así interactuar con su modelo. En particular, se le pide:

- Guardar el código de esta sección en el archivo `main.py`. Note que ejecutar `python main.py` debería levantar el servidor en el puerto por defecto.
- Defina `GET` con ruta tipo *home* que describa brevemente su modelo, el problema que intenta resolver, su entrada y salida.
- Defina un `POST` a la ruta `/potabilidad/` donde utilice su mejor optimizado para predecir si una medición de agua es o no potable. Por ejemplo, una llamada de esta ruta con un *body*:

```json
{
   "ph":10.316400384553162,
   "Hardness":217.2668424334475,
   "Solids":10676.508475429378,
   "Chloramines":3.445514571005745,
   "Sulfate":397.7549459751925,
   "Conductivity":492.20647361771086,
   "Organic_carbon":12.812732207582542,
   "Trihalomethanes":72.28192021570328,
   "Turbidity":3.4073494284238364
}
```

Su servidor debería retornar una respuesta HTML con código 200 con:


```json
{
  "potabilidad": 0 # respuesta puede variar según el clasificador que entrenen
}
```

**`HINT:` Recuerde que puede utilizar [http://localhost:8000/docs](http://localhost:8000/docs) para hacer un `POST`.**

In [ ]:
!pip install "fastapi[all]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.1/442.1 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.6/442.6 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 11.8 MB/s eta 0:00:00


In [ ]:
# main.py

from fastapi import FastAPI
import joblib
import uvicorn

# init app
app = FastAPI()

# Cargamos el modelo previamente entrenado
model = joblib.load("/content/outputs/models/best_model.pkl")

# Ruta principal
@app.get("/")  # Ruta home
async def home():
    return {
        "Hello": "Bienvenido a la API de predicción de potabilidad del agua",
        "Description": "Este modelo predice si el agua es potable (1) o no (0) basándose en mediciones físico-químicas.",
        "Input": {
            "ph": "pH del agua (acidez o alcalinidad)",
            "Hardness": "Dureza del agua",
            "Solids": "Sólidos disueltos totales (mg/L)",
            "Chloramines": "Cantidad de cloraminas (mg/L)",
            "Sulfate": "Concentración de sulfato (mg/L)",
            "Conductivity": "Conductividad del agua (μS/cm)",
            "Organic_carbon": "Carbono orgánico total (mg/L)",
            "Trihalomethanes": "Trihalometanos (ug/L)",
            "Turbidity": "Turbidez (NTU)"
        },
        "Output": "0 (no potable) o 1 (potable)"
    }

# Ruta para predicción
@app.post("/potabilidad/")
async def predict(
    ph: float,
    Hardness: float,
    Solids: float,
    Chloramines: float,
    Sulfate: float,
    Conductivity: float,
    Organic_carbon: float,
    Trihalomethanes: float,
    Turbidity: float
):
    """
    Predicción de la potabilidad del agua basada en parámetros físico-químicos.
    """
    features = [[
        ph,
        Hardness,
        Solids,
        Chloramines,
        Sulfate,
        Conductivity,
        Organic_carbon,
        Trihalomethanes,
        Turbidity
    ]]

    prediction = model.predict(features)[0]

    return {"potabilidad": int(prediction)}

# **3. Docker (2 puntos)**

<div align="center">
  <img src="https://miro.medium.com/v2/resize:fit:1400/1*9rafh2W0rbRJIKJzqYc8yA.gif" width="500">
</div>

Tras el éxito de su aplicación web para generar la salida, Smapina le solicita que genere un contenedor para poder ejecutarla en cualquier computador de la empresa de agua potable.

## **3.1 Creación de Container (1 punto)**

Cree un Dockerfile que use una imagen base de Python, copie los archivos del proyecto e instale las dependencias desde un `requirements.txt`. Con esto, construya y ejecute el contenedor Docker para la API configurada anteriormente. Entregue el código fuente (incluyendo `main.py`, `requirements.txt`, y `Dockerfile`) y la imagen Docker de la aplicación. Para la dockerización, asegúrese de cumplir con los siguientes puntos:

1. **Generar un archivo `.dockerignore`** que ignore carpetas y archivos innecesarios dentro del contenedor.
2. **Configurar un volumen** que permita la persistencia de los datos en una ruta local del computador.
3. **Exponer el puerto** para acceder a la ruta de la API sin tener que entrar al contenedor directamente.
4. **Incluir imágenes en el notebook** que muestren la ejecución del contenedor y los resultados obtenidos.
5. **Revisar y comentar los recursos utilizados por el contenedor**. Analice si los contenedores son livianos en términos de recursos.

## **3.2 Preguntas de Smapina (1 punto)**
Tras haber experimentado con Docker, Smapina desea profundizar más en el tema y decide realizarle las siguientes consultas:

- ¿Cómo se diferencia Docker de una máquina virtual (VM)?
- ¿Cuál es la diferencia entre usar Docker y ejecutar la aplicación directamente en el sistema local?
- ¿Cómo asegura Docker la consistencia entre diferentes entornos de desarrollo y producción?
- ¿Cómo se gestionan los volúmenes en Docker para la persistencia de datos?
- ¿Qué son Dockerfile y docker-compose.yml, y cuál es su propósito?

A continuación los distintos archivos pedidos: main.py, dockerfile, requirements.txt y .dockerignore. Estos son ejecutables en consola.

In [ ]:
# main.py

from fastapi import FastAPI
import joblib
import uvicorn

# init app
app = FastAPI()

# Cargamos el modelo previamente entrenado
model = joblib.load("/content/outputs/models/best_model.pkl")

# Ruta principal
@app.get("/")  # Ruta home
async def home():
    return {
        "Hello": "Bienvenido a la API de predicción de potabilidad del agua",
        "Description": "Este modelo predice si el agua es potable (1) o no (0) basándose en mediciones físico-químicas.",
        "Input": {
            "ph": "pH del agua (acidez o alcalinidad)",
            "Hardness": "Dureza del agua",
            "Solids": "Sólidos disueltos totales (mg/L)",
            "Chloramines": "Cantidad de cloraminas (mg/L)",
            "Sulfate": "Concentración de sulfato (mg/L)",
            "Conductivity": "Conductividad del agua (μS/cm)",
            "Organic_carbon": "Carbono orgánico total (mg/L)",
            "Trihalomethanes": "Trihalometanos (ug/L)",
            "Turbidity": "Turbidez (NTU)"
        },
        "Output": "0 (no potable) o 1 (potable)"
    }

# Ruta para predicción
@app.post("/potabilidad/")
async def predict(
    ph: float,
    Hardness: float,
    Solids: float,
    Chloramines: float,
    Sulfate: float,
    Conductivity: float,
    Organic_carbon: float,
    Trihalomethanes: float,
    Turbidity: float
):
    """
    Predicción de la potabilidad del agua basada en parámetros físico-químicos.
    """
    features = [[
        ph,
        Hardness,
        Solids,
        Chloramines,
        Sulfate,
        Conductivity,
        Organic_carbon,
        Trihalomethanes,
        Turbidity
    ]]

    prediction = model.predict(features)[0]

    return {"potabilidad": int(prediction)}

In [ ]:
# Dockerfile
# Usa una imagen base de Python
FROM python:3.9-slim

# Directorio de trabajo dentro del contenedor
WORKDIR /app

# Copiar archivos necesarios
COPY main.py .
COPY requirements.txt .

# Instalar dependencias
RUN pip install --no-cache-dir -r requirements.txt

# Exponer el puerto donde corre la aplicación FastAPI
EXPOSE 8000

# Comando para ejecutar la aplicación FastAPI dentro del contenedor
CMD ["uvicorn", "main:app", "--port", "8000"]

In [ ]:
# requirements.txt
fastapi
uvicorn
pydantic
scikit-learn

In [ ]:
# .dockerignore
__pycache__
*.pyc
*.log
models/
miruns/
plots/
README.md

1. Docker y las máquinas virtuales (VMs) son tecnologías de virtualización, pero se distinguen en su enfoque y arquitectura:

- Máquinas virtuales: Simulan un sistema operativo completo. Cada MV incluye su propio kernel y sistema operativo, lo que las hace más pesadas en términos de uso de recursos. Esto puede llevar a tiempos de inicio más largos y un mayor consumo de memoria y almacenamiento.
- Docker: Utiliza contenedores que comparten el kernel del sistema operativo en el que se ejecutan, lo que los hace más ligeros y rápidos de iniciar. Los contenedores están diseñados para ejecutar aplicaciones de manera aislada con solo las dependencias necesarias, reduciendo significativamente la sobrecarga de recursos en comparación con las MVs.
2. Ejecutar una aplicación directamente en el sistema local depende de la configuración específica de ese entorno, incluyendo versiones de paquetes, software y dependencias. Esto puede ocasionar problemas como:

- Inconsistencias: La aplicación puede comportarse de manera diferente en distintos sistemas debido a diferencias en las versiones de las dependencias o configuraciones.
- Compatibilidad: Cambios en el sistema local pueden interferir con el funcionamiento del script o producto.
  Con Docker, las aplicaciones y sus dependencias están encapsuladas en un contenedor, asegurando que siempre se ejecuten de manera consistente en cualquier entorno que tenga Docker instalado, independientemente del sistema anfitrión.
3. Docker utiliza imágenes de contenedor para encapsular las dependencias, configuraciones y el propio código de la aplicación. Estas imágenes se crean a partir de un Dockerfile que especifica exactamente cómo construir la aplicación.
Pueden probarse en el entorno de desarrollo y luego desplegarse en producción garantizando replicabilidad. Además, herramientas como docker-compose facilitan la definición y gestión de aplicaciones que requieren múltiples servicios que pueden tener problemas de compatibilidad asegurando que todo el sistema funcione.
4. Los volúmenes en Docker se utilizan para almacenar datos que necesitan persistir más allá del ciclo de vida de un contenedor. Esto incluye bases de datos, especificacions de configuraciones y archivos generados al ejecutar. Esto permite persistencia de datos incluso si el contenedor se detiene o elimina, facilita compartir datos entre contenedores y respaldar o migrar datos.
Los volúmenes se configuran en el comando docker run o en un archivo docker-compose.yml.
5. Un dockerfile es un archivo de texto que contiene las instrucciones para construir una imagen Docker.
Define el sistema operativo base, las dependencias necesarias, los archivos a copiar y los comandos a ejecutar. Esto tiene el fin de automatizar y estandarizar la creación de imágenes Docker garantizando reproductibilidad y consistencia.
docker-compose.yml es un archivo YAML que permite definir y gestionar aplicaciones multi-contenedor.
Especifica servicios, redes y volúmenes necesarios para la aplicación y la interacción entre los contenedores.


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<div align="center">
  <img src="https://i.pinimg.com/originals/84/5d/f1/845df1aefc6a5e37ae575327a0cc6e43.gif" width="500">
</div>